<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:


      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 10:03:08
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 87.91 K (0.62%)
Current PnL: -16.66 L (-11.21%)
CY Booked + Current PnL: -5.63 L (-3.79%)
-------------------
Total profit:  2.26 L
Total loss:  -18.92 L
-------------------
Total Booked + Current PnL: 21.35 L (17.54%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.74%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.56 L (57.92%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-11.34,67.0,M-LC,4.26,220480.0,19000.0,8224.0,0.88,9.43,3.73,13.51,60.0,2.31,1.57,23.22,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.09,60.0,H-LC,11.08,173723.0,17546.0,8547.0,1.02,11.23,4.92,16.71,22.0,2.05,1.24,30.63,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.98,62.0,M-SC,4.41,88752.0,-12025.0,12115.0,0.80,-11.93,13.65,0.09,245.0,-0.99,0.63,16.60,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,69.0,H-MC,5.68,114337.0,13833.0,16808.0,-0.49,13.76,14.70,30.49,79.0,0.82,0.81,50.63,MH,ATH,METALS
57,RELIANCE,1533.00,-12.16,58.0,H-LC,5.59,219976.0,9610.0,19182.0,0.77,4.57,8.72,13.68,37.0,0.50,1.57,21.85,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.00,-20.88,43.0,H-MC,2.23,136131.0,343.0,42405.0,-0.47,0.25,31.15,31.48,107.0,0.01,0.97,12.40,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,60.0,H-MC,5.80,180407.0,599.0,111816.0,-0.59,0.33,61.98,62.52,88.0,0.01,1.29,34.53,XR,NTT,BANKS
52,PGHH,17973.08,-30.68,53.0,H-MC,4.37,200700.0,-120.0,68900.0,0.99,-0.06,34.33,34.25,80.0,-0.00,1.43,5.13,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.03,37.0,H-LC,5.61,298903.0,-16939.0,129604.0,-0.67,-5.36,43.36,35.67,5.0,-0.13,2.13,7.75,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.32,45.0,H-LC,8.16,218856.0,-32912.0,86120.0,-0.62,-13.07,39.35,21.13,27.0,-0.38,1.56,17.72,X40,ATH,PAINTS
20,CIPLA,1795.00,-20.47,49.0,H-LC,5.79,212542.0,8042.0,33369.0,0.20,3.93,15.70,20.25,10.0,0.24,1.51,13.01,X40N,BTT,PHARMA
42,ITC,452.00,-37.92,51.0,H-LC,2.89,200979.0,841.0,20058.0,-0.40,0.42,9.98,10.44,4.0,0.04,1.43,6.01,X40,NTT,FMCG
44,JIOFIN,387.00,0.31,52.0,H-LC,13.40,272620.0,5255.0,60194.0,0.52,1.97,22.08,24.48,48.0,0.09,1.94,58.02,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-1.73,65.0,H-LC,1.00,243405.0,-2746.0,81954.0,1.26,-1.12,33.67,32.18,16.0,-0.03,1.73,34.53,X200,ATH,IT
42,ITC,452.00,-37.92,51.0,H-LC,2.89,200979.0,841.0,20058.0,-0.40,0.42,9.98,10.44,4.0,0.04,1.43,6.01,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.60,56.0,H-LC,4.12,281362.0,15936.0,40854.0,-0.83,6.00,14.52,21.40,11.0,0.39,2.00,13.52,XY25,NTT,FMCG
57,RELIANCE,1533.00,-12.16,58.0,H-LC,5.59,219976.0,9610.0,19182.0,0.77,4.57,8.72,13.68,37.0,0.50,1.57,21.85,XY25,NTT,REFINERIES
84,VBL,671.64,-17.03,37.0,H-LC,5.61,298903.0,-16939.0,129604.0,-0.67,-5.36,43.36,35.67,5.0,-0.13,2.13,7.75,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,63.09,69.0,H-SC,8.55,141214.0,-41561.0,152624.0,5.21,-22.74,108.08,60.77,125.0,-0.27,1.01,26.48,XR,NTT,JEWELLERY
56,RELAXO,1176.00,-31.36,59.0,H-SC,1.75,93539.0,-51621.0,127550.0,3.45,-35.56,136.36,52.31,136.0,-0.40,0.67,25.21,X40N,NTT,FOOTWEAR
77,TRIDENT,48.00,-4.75,65.0,M-SC,5.33,74051.0,-18204.0,44512.0,3.38,-19.73,60.11,28.51,224.0,-0.41,0.53,26.71,XR,NTT,TEXTILES
17,CAMPUS,393.00,-24.03,61.0,M-SC,1.50,158154.0,-9621.0,65460.0,2.45,-5.73,41.39,33.28,210.0,-0.15,1.13,22.58,XY24,NTT,FOOTWEAR
72,TANLA,1963.11,-19.51,69.0,H-SC,10.39,215730.0,-55031.0,384992.0,2.29,-20.32,178.46,121.86,133.0,-0.14,1.54,68.76,AR,ATH,IT


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,38.71,49.0,M-SC,8.41,183128.0,-79090.0,148920.0,-2.59,-30.16,81.32,26.63,229.0,-0.53,1.30,13.86,XY24,NTT,MISC
65,SHALBY,327.00,1061.08,64.0,M-SC,2.55,158004.0,-23139.0,68605.0,-2.03,-12.77,43.42,25.10,235.0,-0.34,1.13,26.62,XY24,NTT,HEALTHCARE
79,UJJIVANSFB,60.00,96.03,56.0,H-SC,12.42,124119.0,-18360.0,37881.0,-1.61,-12.89,30.52,13.70,163.0,-0.48,0.88,46.49,OX40N,NTT,BANKS
37,INDIAMART,4850.92,-51.28,47.0,H-SC,12.13,136104.0,12768.0,120996.0,-1.58,10.35,88.90,108.45,119.0,0.11,0.97,35.02,AR,ATH,MISC
23,COLPAL,3767.14,0.50,50.0,H-MC,7.31,182832.0,-28909.0,110997.0,-0.91,-13.65,60.71,38.77,84.0,-0.26,1.30,8.82,XY25,ATH,FMCG


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.98,62.0,M-SC,4.41,88752.0,-12025.0,12115.0,0.80,-11.93,13.65,0.09,245.0,-0.99,0.63,16.60,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,56.0,H-SC,12.42,124119.0,-18360.0,37881.0,-1.61,-12.89,30.52,13.70,163.0,-0.48,0.88,46.49,OX40N,NTT,BANKS
19,CERA,9475.0,-20.05,45.0,H-SC,2.16,145659.0,-30244.0,72261.0,0.22,-17.19,49.61,23.89,149.0,-0.42,1.04,25.80,OX40N,NTT,CERAMICS
45,KANSAINER,340.0,-67.61,59.0,H-SC,1.74,223191.0,-46476.0,82804.0,0.75,-17.23,37.10,13.47,138.0,-0.56,1.59,12.89,XY24,NTT,PAINTS
73,TATAELXSI,9161.0,-19.51,55.0,H-MC,7.77,108994.0,-22841.0,65069.0,0.91,-17.33,59.70,32.03,98.0,-0.35,0.78,22.08,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,59.0,H-LC,12.77,297914.0,-48042.0,122890.0,1.84,-13.89,41.25,21.64,1.0,-0.39,2.12,5.54,X40,ATH,IT
40,INFY,2275.00,-17.09,54.0,H-LC,8.78,323355.0,10467.0,161225.0,0.64,3.35,49.86,54.87,3.0,0.06,2.30,10.22,X40,BTT,IT
42,ITC,452.00,-37.92,51.0,H-LC,2.89,200979.0,841.0,20058.0,-0.40,0.42,9.98,10.44,4.0,0.04,1.43,6.01,X40,NTT,FMCG
84,VBL,671.64,-17.03,37.0,H-LC,5.61,298903.0,-16939.0,129604.0,-0.67,-5.36,43.36,35.67,5.0,-0.13,2.13,7.75,X40N,ATH,FMCG
1,ABB,7934.00,-38.02,65.0,H-LC,11.09,258960.0,-2659.0,121867.0,1.05,-1.02,47.06,45.57,7.0,-0.02,1.85,10.19,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,65.0,M-MC,6.96,228664.0,3702.0,163952.0,-0.00,1.65,71.70,74.53,192.0,0.02,1.63,34.43,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,10.86,57.0,H-MC,3.68,191330.0,26660.0,41308.0,1.28,16.19,21.59,41.28,89.0,0.65,1.36,25.46,X40N,ATH,AC
30,HAVELLS,2069.16,1.19,65.0,H-MC,1.82,250879.0,3148.0,71902.0,0.81,1.27,28.66,30.30,92.0,0.04,1.79,14.73,X40,ATH,ELECTRICAL
85,VOLTAS,1918.49,0.88,58.0,H-MC,2.69,211635.0,19893.0,76146.0,-0.35,10.37,35.98,50.08,99.0,0.26,1.51,17.88,XY25,ATH,AC
18,CAMS,5211.76,-3.83,55.0,H-SC,4.45,109301.0,7297.0,36627.0,1.13,7.15,33.51,43.06,122.0,0.20,0.78,25.49,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,69.0,H-MC,5.68,114337.0,13833.0,16808.0,-0.49,13.76,14.70,30.49,79.0,0.82,0.81,50.63,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.15,60.0,M-SC,0.62,102900.0,11402.0,67348.0,2.11,12.46,65.45,86.07,223.0,0.17,0.73,49.43,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.28,47.0,H-SC,12.13,136104.0,12768.0,120996.0,-1.58,10.35,88.90,108.45,119.0,0.11,0.97,35.02,AR,ATH,MISC
87,WIPRO,420.00,-10.16,57.0,M-LC,5.55,157740.0,6795.0,102657.0,1.25,4.50,65.08,72.51,53.0,0.07,1.12,10.80,XR,NTT,IT
36,IEX,219.00,-28.54,59.0,H-SC,17.25,203876.0,3922.0,93314.0,2.06,1.96,45.77,48.63,115.0,0.04,1.45,13.74,XR,NTT,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.92,87072.0,-26477.0,66532.0,2.09,-23.32,76.41,35.27,268.0,-0.40,0.62,113.11,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.50,50.0,H-SC,13.36,89797.0,-11166.0,105700.0,1.31,-11.06,117.71,93.63,148.0,-0.11,0.64,33.75,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.99,57.0,H-SC,12.92,95535.0,-2445.0,24046.0,-0.45,-2.50,25.17,22.05,152.0,-0.10,0.68,38.87,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,69.0,H-MC,5.68,114337.0,13833.0,16808.0,-0.49,13.76,14.70,30.49,79.0,0.82,0.81,50.63,MH,ATH,METALS
8,ATULAUTO,844.00,3856.76,64.0,M-SC,6.48,123334.0,-24034.0,75850.0,0.92,-16.31,61.50,35.16,236.0,-0.32,0.88,25.97,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-41.65,55.0,L-SC,41.92,87072.0,-26477.0,66532.0,2.09,-23.32,76.41,35.27,268.0,-0.40,0.62,113.11,XR,NTT,HOTELS
8,ATULAUTO,844.0,3856.76,64.0,M-SC,6.48,123334.0,-24034.0,75850.0,0.92,-16.31,61.50,35.16,236.0,-0.32,0.88,25.97,XY24,NTT,AUTO
61,SAMMAANCAP,326.0,-171.59,63.0,M-SC,2.61,82320.0,-19890.0,113281.0,1.02,-19.46,137.61,91.37,208.0,-0.18,0.59,33.63,XY25,NTT,FINANCE
76,TITAGARH,1548.0,0.96,69.0,H-SC,4.10,165436.0,-26579.0,105465.0,-0.31,-13.84,63.75,41.08,158.0,-0.25,1.18,40.19,XY24,NTT,ENGINEERING
54,RAJESHEXPO,518.0,1889.90,67.0,L-SC,4.42,54298.0,-82879.0,82973.0,1.93,-60.42,152.81,0.07,267.0,-1.00,0.39,35.04,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,42.76
2,50,73.16


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.37
LC    33.99
MC    22.67
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.46
X40      15.01
X40N     12.79
XR       11.43
XY25      9.59
AR        8.58
OX40N     8.05
X200      1.73
MH        1.69
X5K       1.43
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.50
H-LC    25.87
H-MC    19.77
M-SC    13.33
M-LC     7.07
M-MC     2.60
L-SC     1.54
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.24,-13.09,74.75
FMCG,11.56,-4.23,40.70
FINANCE,9.51,-15.31,60.47
BANKS,7.62,-12.12,63.75
MISC,7.34,-11.55,73.59
PAINTS,5.72,-15.00,32.14
ELECTRICAL,5.63,-4.45,62.01
AC,3.57,3.28,29.47
INSURANCE,3.35,-6.75,43.37


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2850025.0,22
XR,1306881.0,14
AR,1295804.0,9
X40,740633.0,10
X40N,617704.0,10
OX40N,518777.0,10
XY25,405321.0,7
SR,243302.0,2
X5K,117353.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3416698.0,29
M-SC,1376740.0,17
H-LC,1242553.0,15
H-MC,1177532.0,15
M-LC,386243.0,5
M-MC,312890.0,2
L-SC,246098.0,3
L-MC,60267.0,1
L-LC,36637.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1032553.0      6
M-SC       XY24       799905.0      7
H-SC       AR         780950.0      5
           XR         757156.0      7
H-MC       XY24       486474.0      4
H-LC       X40        461929.0      5
           AR         353748.0      2
M-MC       XY24       312890.0      2
H-SC       X40N       292584.0      4
           OX40N      249057.0      4
           SR         243302.0      2
H-LC       X40N       224692.0      4
H-MC       X40        209794.0      4
           XY25       187143.0      2
L-SC       XR         163125.0      2
M-SC       AR         161106.0      2
M-LC       XY24       158009.0      2
M-SC       OX40N      121678.0      4
M-LC       X5K        117353.0      1
M-SC       XY25       113281.0      1
           XR         111860.0      2
H-MC       XR         111816.0      1
M-LC       XR         102657.0      1
H-MC       X40N       100428.0      2
L-SC       OX40N       82973.0      1
H-LC       X200        81954.0      1
M-SC       X40         68910.0      1
H-MC       OX40N       65069.0      1
H-SC       MH          61096.0      1
L-MC       XR          60267.0      1
H-LC       XY24        60194.0      1
           XY25        60036.0      2
L-LC       XY25        36637.0      1
H-MC       MH          16808.0      1
M-LC       XY25         8224.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
